In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    confusion_matrix, cohen_kappa_score, matthews_corrcoef
)
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
#from keras.models import Sequential
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
from keras.layers import Dense, Conv1D, MaxPool1D, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Reshape, SimpleRNN

In [ ]:
# Load Training Data
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Peptides/Multi Fusion 0.3/LSA+ PAAC merge.csv')
df['target'].value_counts()

,count
target,
0,2425
1,2425


In [ ]:
target = 'target'

Y = df[target]
X= df.drop(target, axis=1)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split (X,Y,test_size=0.3,random_state=42 )


**Train part**

In [ ]:
# Reshape X_train for Conv1D
X_train1 = X_train.to_numpy().reshape(X_train.shape[0], X_train.shape[1], 1)
Y_train1 = Y_train.to_numpy()

# Define KFold for cross-validation
kf = KFold(n_splits=5, random_state=1, shuffle=True)

# Iterate over KFold splits

for train_index, val_index in kf.split(X_train1):


    # Split the data
    X_train_fold, X_val_fold = X_train1[train_index], X_train1[val_index]
    Y_train_fold, Y_val_fold = Y_train1[train_index], Y_train1[val_index]

**Test part**


In [ ]:
X_test1 = X_test.to_numpy().reshape(X_test.shape[0], X_test.shape[1], 1)
Y_test1 = Y_test.to_numpy()

# **CNN**

In [ ]:
# Build the CNN model
cnn = Sequential()
cnn.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train1.shape[1], 1)))
cnn.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
cnn.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
cnn.add(MaxPool1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(128, activation='relu'))
cnn.add(Dense(64, activation='relu'))
cnn.add(Dense(1, activation='sigmoid'))

# Compile the CNN model
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on the training dataset
cnn.fit(X_train1, Y_train1, epochs=40, batch_size=64)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 11s 37ms/step - accuracy: 0.6383 - loss: 0.6211
Epoch 2/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.7874 - loss: 0.4628
Epoch 3/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8589 - loss: 0.3342
Epoch 4/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9010 - loss: 0.2474
Epoch 5/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9418 - loss: 0.1627
Epoch 6/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9710 - loss: 0.0832
Epoch 7/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9664 - loss: 0.0908
Epoch 8/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9930 - loss: 0.0259
Epoch 9/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9955 - loss: 0.0194
Epoch 10/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9539 - loss: 0.1260
Epoch 11/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9982 - loss: 0.0090
Epoch 12/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - 

**CNN Train**

In [ ]:
# Evaluate the model on the training set
train_pred = cnn.predict(X_train_fold)
train_pred_classes = np.round(train_pred).astype(int)

# Performance metrics for training set
train_accuracy = accuracy_score(Y_train_fold, train_pred_classes)
train_mcc = matthews_corrcoef(Y_train_fold, train_pred_classes)
train_kappa = cohen_kappa_score(Y_train_fold, train_pred_classes)
train_precision = precision_score(Y_train_fold, train_pred_classes)
train_f1 = f1_score(Y_train_fold, train_pred_classes)
train_cm = confusion_matrix(Y_train_fold, train_pred_classes)
train_specificity = train_cm[0, 0] / (train_cm[0, 0] + train_cm[0, 1])
train_sensitivity = train_cm[1, 1] / (train_cm[1, 0] + train_cm[1, 1])

print("Training Metrics:")
print(f"Accuracy: {train_accuracy}")
print(f"Matthews Correlation Coefficient (MCC): {train_mcc}")
print(f"Kappa: {train_kappa}")
print(f"Precision: {train_precision}")
print(f"F1 Score: {train_f1}")
print(f"Specificity: {train_specificity}")
print(f"Sensitivity: {train_sensitivity}")

85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Training Metrics:
Accuracy: 1.0
Matthews Correlation Coefficient (MCC): 1.0
Kappa: 1.0
Precision: 1.0
F1 Score: 1.0
Specificity: 1.0
Sensitivity: 1.0


**CNN Test**

In [ ]:
# Predict on the test set
test_pred = cnn.predict(X_test1)
test_pred_classes = (test_pred > 0.5).astype(int)


# Performance metrics for test set
test_accuracy = accuracy_score(Y_test1, test_pred_classes)
test_mcc = matthews_corrcoef(Y_test1, test_pred_classes)
test_kappa = cohen_kappa_score(Y_test1, test_pred_classes)
test_precision = precision_score(Y_test1, test_pred_classes)
test_f1 = f1_score(Y_test1, test_pred_classes)
test_cm = confusion_matrix(Y_test1, test_pred_classes)
test_specificity = test_cm[0, 0] / (test_cm[0, 0] + test_cm[0, 1])
test_sensitivity = test_cm[1, 1] / (test_cm[1, 0] + test_cm[1, 1])

print("\nTest Metrics:")
print(f"Accuracy: {test_accuracy}")
print(f"Matthews Correlation Coefficient (MCC): {test_mcc}")
print(f"Kappa: {test_kappa}")
print(f"Precision: {test_precision}")
print(f"F1 Score: {test_f1}")
print(f"Specificity: {test_specificity}")
print(f"Sensitivity: {test_sensitivity}")

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step

Test Metrics:
Accuracy: 0.9457044673539519
Matthews Correlation Coefficient (MCC): 0.8913086941663508
Kappa: 0.8912402743570407
Precision: 0.9495677233429395
F1 Score: 0.9434502505368647
Specificity: 0.9534574468085106
Sensitivity: 0.9374110953058321


# **BiLSTM**

In [ ]:
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout


# Build the BiLSTM model
bilstm_model = Sequential()
bilstm_model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(X_train1.shape[1], 1)))
bilstm_model.add(Bidirectional(LSTM(64)))
bilstm_model.add(Dropout(0.5))  # Helps prevent overfitting
bilstm_model.add(Dense(128, activation='relu'))
bilstm_model.add(Dense(64, activation='relu'))
bilstm_model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the BiLSTM model
bilstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
bilstm_model.fit(X_train1, Y_train1, epochs=40, batch_size=64, verbose=1)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 8s 28ms/step - accuracy: 0.5054 - loss: 0.6944
Epoch 2/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.5774 - loss: 0.6795
Epoch 3/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.6076 - loss: 0.6654
Epoch 4/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.6210 - loss: 0.6483
Epoch 5/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.6327 - loss: 0.6431
Epoch 6/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.6275 - loss: 0.6490
Epoch 7/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.6525 - loss: 0.6302
Epoch 8/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.6604 - loss: 0.6295
Epoch 9/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.6613 - loss: 0.6148
Epoch 10/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.6975 - loss: 0.5824
Epoch 11/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.6775 - loss: 0.5949
Epoch 12/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy:

**BiLSTM Train**

In [ ]:
# Evaluate the BiLSTM model on training data
train_pred = bilstm_model.predict(X_train_fold)
train_pred_classes =  np.round(train_pred).astype(int)


# Performance metrics for training set
train_accuracy = accuracy_score(Y_train_fold, train_pred_classes)
train_mcc = matthews_corrcoef(Y_train_fold, train_pred_classes)
train_kappa = cohen_kappa_score(Y_train_fold, train_pred_classes)
train_precision = precision_score(Y_train_fold, train_pred_classes)
train_f1 = f1_score(Y_train_fold, train_pred_classes)
train_cm = confusion_matrix(Y_train_fold, train_pred_classes)
train_specificity = train_cm[0, 0] / (train_cm[0, 0] + train_cm[0, 1])
train_sensitivity = train_cm[1, 1] / (train_cm[1, 0] + train_cm[1, 1])

print("\nTraining Metrics:")
print(f"Accuracy: {train_accuracy}")
print(f"Matthews Correlation Coefficient (MCC): {train_mcc}")
print(f"Kappa: {train_kappa}")
print(f"Precision: {train_precision}")
print(f"F1 Score: {train_f1}")
print(f"Specificity: {train_specificity}")
print(f"Sensitivity: {train_sensitivity}")

85/85 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step

Training Metrics:
Accuracy: 0.7650957290132547
Matthews Correlation Coefficient (MCC): 0.5302956647335665
Kappa: 0.5301662360057807
Precision: 0.7705660377358491
F1 Score: 0.7619402985074627
Specificity: 0.7766348273328435
Sensitivity: 0.7535055350553506


**BiLSTM Test**

In [ ]:
# Evaluate the BiLSTM model on test data
test_pred = bilstm_model.predict(X_test1)
test_pred_classes = (test_pred > 0.5).astype(int)


# Performance metrics for test set
test_accuracy = accuracy_score(Y_test1, test_pred_classes)
test_mcc = matthews_corrcoef(Y_test1, test_pred_classes)
test_kappa = cohen_kappa_score(Y_test1, test_pred_classes)
test_precision = precision_score(Y_test1, test_pred_classes)
test_f1 = f1_score(Y_test1, test_pred_classes)
test_cm = confusion_matrix(Y_test1, test_pred_classes)
test_specificity = test_cm[0, 0] / (test_cm[0, 0] + test_cm[0, 1])
test_sensitivity = test_cm[1, 1] / (test_cm[1, 0] + test_cm[1, 1])

print("\nTest Metrics:")
print(f"Accuracy: {test_accuracy}")
print(f"Matthews Correlation Coefficient (MCC): {test_mcc}")
print(f"Kappa: {test_kappa}")
print(f"Precision: {test_precision}")
print(f"F1 Score: {test_f1}")
print(f"Specificity: {test_specificity}")
print(f"Sensitivity: {test_sensitivity}")

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step

Test Metrics:
Accuracy: 0.7457044673539519
Matthews Correlation Coefficient (MCC): 0.49125266897991504
Kappa: 0.49116160900115124
Precision: 0.7322175732217573
F1 Score: 0.7394366197183099
Specificity: 0.7446808510638298
Sensitivity: 0.7467994310099573


# **RNN**

In [ ]:
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout



# Build the RNN model
rnn_model = Sequential()
rnn_model.add(SimpleRNN(64, return_sequences=True, input_shape=(X_train1.shape[1], 1), activation='relu'))
rnn_model.add(SimpleRNN(64, activation='relu'))
rnn_model.add(Dropout(0.5))  # Helps prevent overfitting
rnn_model.add(Dense(128, activation='relu'))
rnn_model.add(Dense(64, activation='relu'))
rnn_model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the RNN model
rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the RNN model
rnn_model.fit(X_train1, Y_train1, epochs=40, batch_size=64, verbose=1)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 9s 70ms/step - accuracy: 0.5963 - loss: 0.6649
Epoch 2/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.6862 - loss: 0.6148
Epoch 3/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.6982 - loss: 0.5867
Epoch 4/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7267 - loss: 0.5491
Epoch 5/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7264 - loss: 0.5508
Epoch 6/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7699 - loss: 0.4937
Epoch 7/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.7666 - loss: 0.4942
Epoch 8/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.7526 - loss: 0.5023
Epoch 9/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7850 - loss: 0.4717
Epoch 10/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.7836 - loss: 0.4816
Epoch 11/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.7201 - loss: 0.5527
Epoch 12/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy:

**RNN Train**

In [ ]:
# Evaluate the RNN model on the training set
train_pred = rnn_model.predict(X_train_fold)
train_pred_classes =  np.round(train_pred).astype(int)


# Performance metrics for training set
train_accuracy = accuracy_score(Y_train_fold, train_pred_classes)
train_mcc = matthews_corrcoef(Y_train_fold, train_pred_classes)
train_kappa = cohen_kappa_score(Y_train_fold, train_pred_classes)
train_precision = precision_score(Y_train_fold, train_pred_classes)
train_f1 = f1_score(Y_train_fold, train_pred_classes)
train_cm = confusion_matrix(Y_train_fold, train_pred_classes)
train_specificity = train_cm[0, 0] / (train_cm[0, 0] + train_cm[0, 1])
train_sensitivity = train_cm[1, 1] / (train_cm[1, 0] + train_cm[1, 1])

print("\nTraining Metrics:")
print(f"Accuracy: {train_accuracy}")
print(f"Matthews Correlation Coefficient (MCC): {train_mcc}")
print(f"Kappa: {train_kappa}")
print(f"Precision: {train_precision}")
print(f"F1 Score: {train_f1}")
print(f"Specificity: {train_specificity}")
print(f"Sensitivity: {train_sensitivity}")

85/85 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step

Training Metrics:
Accuracy: 0.9436671575846833
Matthews Correlation Coefficient (MCC): 0.8874652665621816
Kappa: 0.8873379806357579
Precision: 0.9361393323657474
F1 Score: 0.9440175631174533
Specificity: 0.9353416605437178
Sensitivity: 0.9520295202952029


**RNN Test**

In [ ]:
# Evaluate the RNN model on the test set
test_pred = rnn_model.predict(X_test1)
test_pred_classes = (test_pred > 0.5).astype(int)


# Performance metrics for test set
test_accuracy = accuracy_score(Y_test1, test_pred_classes)
test_mcc = matthews_corrcoef(Y_test1, test_pred_classes)
test_kappa = cohen_kappa_score(Y_test1, test_pred_classes)
test_precision = precision_score(Y_test1, test_pred_classes)
test_f1 = f1_score(Y_test1, test_pred_classes)
test_cm = confusion_matrix(Y_test1, test_pred_classes)
test_specificity = test_cm[0, 0] / (test_cm[0, 0] + test_cm[0, 1])
test_sensitivity = test_cm[1, 1] / (test_cm[1, 0] + test_cm[1, 1])

print("\nTest Metrics:")
print(f"Accuracy: {test_accuracy}")
print(f"Matthews Correlation Coefficient (MCC): {test_mcc}")
print(f"Kappa: {test_kappa}")
print(f"Precision: {test_precision}")
print(f"F1 Score: {test_f1}")
print(f"Specificity: {test_specificity}")
print(f"Sensitivity: {test_sensitivity}")

46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step

Test Metrics:
Accuracy: 0.8096219931271478
Matthews Correlation Coefficient (MCC): 0.6203902844154423
Kappa: 0.6194993877587628
Precision: 0.7870619946091644
F1 Score: 0.8083044982698961
Specificity: 0.7898936170212766
Sensitivity: 0.8307254623044097


# **GRU**

In [ ]:
from tensorflow.keras.layers import GRU, Dense, Dropout


# Build the GRU model
gru_model = Sequential()
gru_model.add(GRU(64, return_sequences=True, input_shape=(X_train1.shape[1], 1), activation='relu'))
gru_model.add(GRU(64, activation='relu'))
gru_model.add(Dropout(0.5))  # Helps prevent overfitting
gru_model.add(Dense(128, activation='relu'))
gru_model.add(Dense(64, activation='relu'))
gru_model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the GRU model
gru_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the GRU model
gru_model.fit(X_train1, Y_train1, epochs=40, batch_size=64, verbose=1)

Epoch 1/40


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


54/54 ━━━━━━━━━━━━━━━━━━━━ 12s 77ms/step - accuracy: 0.5307 - loss: 0.6907
Epoch 2/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.5702 - loss: 0.6810
Epoch 3/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.5990 - loss: 0.6699
Epoch 4/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.6162 - loss: 0.6617
Epoch 5/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.6233 - loss: 0.6531
Epoch 6/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.6268 - loss: 0.6523
Epoch 7/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.6185 - loss: 0.6454
Epoch 8/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.5959 - loss: 0.6619
Epoch 9/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.6443 - loss: 0.6401
Epoch 10/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.6258 - loss: 0.6575
Epoch 11/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.6065 - loss: 0.6897
Epoch 12/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.6151 - 

 **GRU Train**

In [ ]:
# Evaluate the GRU model on the training set
train_pred = gru_model.predict(X_train_fold)
train_pred_classes =  np.round(train_pred).astype(int)


# Performance metrics for training set
train_accuracy = accuracy_score(Y_train_fold, train_pred_classes)
train_mcc = matthews_corrcoef(Y_train_fold, train_pred_classes)
train_kappa = cohen_kappa_score(Y_train_fold, train_pred_classes)
train_precision = precision_score(Y_train_fold, train_pred_classes)
train_f1 = f1_score(Y_train_fold, train_pred_classes)
train_cm = confusion_matrix(Y_train_fold, train_pred_classes)
train_specificity = train_cm[0, 0] / (train_cm[0, 0] + train_cm[0, 1])
train_sensitivity = train_cm[1, 1] / (train_cm[1, 0] + train_cm[1, 1])

print("\nTraining Metrics:")
print(f"Accuracy: {train_accuracy}")
print(f"Matthews Correlation Coefficient (MCC): {train_mcc}")
print(f"Kappa: {train_kappa}")
print(f"Precision: {train_precision}")
print(f"F1 Score: {train_f1}")
print(f"Specificity: {train_specificity}")
print(f"Sensitivity: {train_sensitivity}")

85/85 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step

Training Metrics:
Accuracy: 0.7540500736377025
Matthews Correlation Coefficient (MCC): 0.5119741030317658
Kappa: 0.5082289457007221
Precision: 0.7261356155365372
F1 Score: 0.767571329157968
Specificity: 0.6943423952975754
Sensitivity: 0.8140221402214022


**GRU Test**

In [ ]:
# Evaluate the GRU model on the test set
test_pred = gru_model.predict(X_test1)
test_pred_classes = (test_pred > 0.5).astype(int)


# Performance metrics for test set
test_accuracy = accuracy_score(Y_test1, test_pred_classes)
test_mcc = matthews_corrcoef(Y_test1, test_pred_classes)
test_kappa = cohen_kappa_score(Y_test1, test_pred_classes)
test_precision = precision_score(Y_test1, test_pred_classes)
test_f1 = f1_score(Y_test1, test_pred_classes)
test_cm = confusion_matrix(Y_test1, test_pred_classes)
test_specificity = test_cm[0, 0] / (test_cm[0, 0] + test_cm[0, 1])
test_sensitivity = test_cm[1, 1] / (test_cm[1, 0] + test_cm[1, 1])

print("\nTest Metrics:")
print(f"Accuracy: {test_accuracy}")
print(f"Matthews Correlation Coefficient (MCC): {test_mcc}")
print(f"Kappa: {test_kappa}")
print(f"Precision: {test_precision}")
print(f"F1 Score: {test_f1}")
print(f"Specificity: {test_specificity}")
print(f"Sensitivity: {test_sensitivity}")

46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step

Test Metrics:
Accuracy: 0.7326460481099656
Matthews Correlation Coefficient (MCC): 0.4712601767110722
Kappa: 0.46708064007381844
Precision: 0.69625
F1 Score: 0.7411842980705257
Specificity: 0.6768617021276596
Sensitivity: 0.7923186344238976


# **CNN+RNN = Deep_NUP_Pred**

In [ ]:
# Build the CNN model
cnn_rnn= Sequential()
cnn_rnn.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train1.shape[1], 1)))
cnn_rnn.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
cnn_rnn.add(Conv1D(filters=128, kernel_size=2, activation='relu'))#new
cnn_rnn.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
cnn_rnn.add(MaxPool1D(pool_size=2))
cnn_rnn.add(Flatten())
cnn_rnn.add(Dense(128, activation='relu'))
cnn_rnn.add(Dense(128, activation='relu'))#new
cnn_rnn.add(Dense(64, activation='relu'))

# Add Simple RNN layer
cnn_rnn.add(Reshape((1, 64)))  # Flatten the output before passing to RNN
cnn_rnn.add(SimpleRNN(128, activation='relu', return_sequences=True))#new
cnn_rnn.add(SimpleRNN(64, activation='relu' ))
cnn_rnn.add(Dense(1, activation='sigmoid'))
# Compile and train the model
cnn_rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn_rnn.fit(X_train1, Y_train1, epochs=40, batch_size=64, verbose=1)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 12s 53ms/step - accuracy: 0.6103 - loss: 0.6463
Epoch 2/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.7649 - loss: 0.5406
Epoch 3/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8461 - loss: 0.3526
Epoch 4/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8760 - loss: 0.2960
Epoch 5/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9272 - loss: 0.1917
Epoch 6/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9649 - loss: 0.1050
Epoch 7/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9551 - loss: 0.1219
Epoch 8/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9852 - loss: 0.0429
Epoch 9/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9925 - loss: 0.0286
Epoch 10/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9713 - loss: 0.0890
Epoch 11/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9961 - loss: 0.0149
Epoch 12/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9969 - 

**Deep_NUP_Pred Train**

In [ ]:
# Evaluate the CNN+RNN model on the training set
train_pred = cnn_rnn.predict(X_train_fold)
train_pred_classes =  np.round(train_pred).astype(int)
# Performance metrics for training set
train_accuracy = accuracy_score(Y_train_fold, train_pred_classes)
train_mcc = matthews_corrcoef(Y_train_fold, train_pred_classes)
train_kappa = cohen_kappa_score(Y_train_fold, train_pred_classes)
train_precision = precision_score(Y_train_fold, train_pred_classes)
train_f1 = f1_score(Y_train_fold, train_pred_classes)
train_cm = confusion_matrix(Y_train_fold, train_pred_classes)
train_specificity = train_cm[0, 0] / (train_cm[0, 0] + train_cm[0, 1])
train_sensitivity = train_cm[1, 1] / (train_cm[1, 0] + train_cm[1, 1])

print("\nTraining Metrics:")
print(f"Accuracy: {train_accuracy}")
print(f"Matthews Correlation Coefficient (MCC): {train_mcc}")
print(f"Kappa: {train_kappa}")
print(f"Precision: {train_precision}")
print(f"F1 Score: {train_f1}")
print(f"Specificity: {train_specificity}")
print(f"Sensitivity: {train_sensitivity}")

85/85 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step

Training Metrics:
Accuracy: 1.0
Matthews Correlation Coefficient (MCC): 1.0
Kappa: 1.0
Precision: 1.0
F1 Score: 1.0
Specificity: 1.0
Sensitivity: 1.0


**Deep_NUP_Pred Test**

In [ ]:
# Evaluate the GRU model on the test set
test_pred = cnn_rnn.predict(X_test1)
test_pred_classes = (test_pred > 0.5).astype(int)


# Performance metrics for test set
test_accuracy = accuracy_score(Y_test1, test_pred_classes)
test_mcc = matthews_corrcoef(Y_test1, test_pred_classes)
test_kappa = cohen_kappa_score(Y_test1, test_pred_classes)
test_precision = precision_score(Y_test1, test_pred_classes)
test_f1 = f1_score(Y_test1, test_pred_classes)
test_cm = confusion_matrix(Y_test1, test_pred_classes)
test_specificity = test_cm[0, 0] / (test_cm[0, 0] + test_cm[0, 1])
test_sensitivity = test_cm[1, 1] / (test_cm[1, 0] + test_cm[1, 1])

print("\nTest Metrics:")
print(f"Accuracy: {test_accuracy}")
print(f"Matthews Correlation Coefficient (MCC): {test_mcc}")
print(f"Kappa: {test_kappa}")
print(f"Precision: {test_precision}")
print(f"F1 Score: {test_f1}")
print(f"Specificity: {test_specificity}")
print(f"Sensitivity: {test_sensitivity}")

46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step

Test Metrics:
Accuracy: 0.954639175257732
Matthews Correlation Coefficient (MCC): 0.9095296528414867
Kappa: 0.9092510447044682
Precision: 0.941747572815534
F1 Score: 0.9536516853932584
Specificity: 0.9441489361702128
Sensitivity: 0.9658605974395448


# **CNN+BiLsTM = CNBLM**

In [ ]:
model = Sequential()

# CNN layers (feature extraction)
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train1.shape[1], 1)))  # Adjust input shape
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Reshape((-1, 128)))  # Adjust the dimension to match the output size from CNN
# Add BiLSTM layer
model.add(Bidirectional(LSTM(64, activation='relu')))  # BiLSTM for capturing bidirectional context
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # For binary classification, change if needed for multi-class

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train1, Y_train1, epochs=40, batch_size=64)

# Evaluate the CNN+BiLSTM model on the training set
X_train_fold_reshaped = X_train_fold.reshape(-1, X_train_fold.shape[1], 1)  # Reshape for CNN input


Epoch 1/40


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


54/54 ━━━━━━━━━━━━━━━━━━━━ 11s 55ms/step - accuracy: 0.5738 - loss: 0.6924
Epoch 2/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.6232 - loss: 0.6499
Epoch 3/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7117 - loss: 0.5574
Epoch 4/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7311 - loss: 0.5288
Epoch 5/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7734 - loss: 0.4781
Epoch 6/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7725 - loss: 0.4744
Epoch 7/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7967 - loss: 0.4373
Epoch 8/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7889 - loss: 0.4453
Epoch 9/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8247 - loss: 0.3805
Epoch 10/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8300 - loss: 0.3593
Epoch 11/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8384 - loss: 0.3652
Epoch 12/40
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8717 - 

**CNBLM Train**

In [ ]:
# Make predictions on the training set
train_pred = model.predict(X_train_fold_reshaped)  # Using the correct model 'model'
train_pred_classes = np.round(train_pred).astype(int)

# Performance metrics for training set
train_accuracy = accuracy_score(Y_train_fold, train_pred_classes)
train_mcc = matthews_corrcoef(Y_train_fold, train_pred_classes)
train_kappa = cohen_kappa_score(Y_train_fold, train_pred_classes)
train_precision = precision_score(Y_train_fold, train_pred_classes)
train_f1 = f1_score(Y_train_fold, train_pred_classes)
train_cm = confusion_matrix(Y_train_fold, train_pred_classes)
train_specificity = train_cm[0, 0] / (train_cm[0, 0] + train_cm[0, 1])
train_sensitivity = train_cm[1, 1] / (train_cm[1, 0] + train_cm[1, 1])

print("\nTraining Metrics:")
print(f"Accuracy: {train_accuracy}")
print(f"Matthews Correlation Coefficient (MCC): {train_mcc}")
print(f"Kappa: {train_kappa}")
print(f"Precision: {train_precision}")
print(f"F1 Score: {train_f1}")
print(f"Specificity: {train_specificity}")
print(f"Sensitivity: {train_sensitivity}")

85/85 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step

Training Metrics:
Accuracy: 0.8394698085419735
Matthews Correlation Coefficient (MCC): 0.7058010959219806
Kappa: 0.6791311817001894
Precision: 0.7663768115942029
F1 Score: 0.8584415584415584
Specificity: 0.7038941954445261
Sensitivity: 0.9756457564575646


**CNBLM Test**

In [ ]:
# Evaluate the GRU model on the test set
test_pred = model.predict(X_test1)
test_pred_classes = (test_pred > 0.5).astype(int)


# Performance metrics for test set
test_accuracy = accuracy_score(Y_test1, test_pred_classes)
test_mcc = matthews_corrcoef(Y_test1, test_pred_classes)
test_kappa = cohen_kappa_score(Y_test1, test_pred_classes)
test_precision = precision_score(Y_test1, test_pred_classes)
test_f1 = f1_score(Y_test1, test_pred_classes)
test_cm = confusion_matrix(Y_test1, test_pred_classes)
test_specificity = test_cm[0, 0] / (test_cm[0, 0] + test_cm[0, 1])
test_sensitivity = test_cm[1, 1] / (test_cm[1, 0] + test_cm[1, 1])

print("\nTest Metrics:")
print(f"Accuracy: {test_accuracy}")
print(f"Matthews Correlation Coefficient (MCC): {test_mcc}")
print(f"Kappa: {test_kappa}")
print(f"Precision: {test_precision}")
print(f"F1 Score: {test_f1}")
print(f"Specificity: {test_specificity}")
print(f"Sensitivity: {test_sensitivity}")

46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step

Test Metrics:
Accuracy: 0.786254295532646
Matthews Correlation Coefficient (MCC): 0.6067713595462567
Kappa: 0.5765175244938621
Precision: 0.7103004291845494
F1 Score: 0.8097859327217125
Specificity: 0.6409574468085106
Sensitivity: 0.9416785206258891
